<a id="1"></a>
# Import data

In [1]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from keras.optimizers import Adam

# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator

from pyspark.sql.functions import col,isnan, when, count
from pyspark.sql.functions import col, create_map, lit
from itertools import chain

WARNING


In [2]:
# Spark Session
conf = SparkConf().setAppName('Predict Loan Payback').setMaster('local[6]') # 6 cores
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

In [3]:
# sc.stop()
sc

<SparkContext master=local[6] appName=Predict Loan Payback>

In [4]:
# Load Data to Spark Dataframe
df = sql_context.read.csv("./data/accepted_2007_to_2018Q4.csv", header=True, inferSchema=True)

In [5]:
# View Schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

<a id="2"></a>
# Preprocessing

In [6]:
df.limit(5).toPandas()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,None,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,None,None,Cash,N,None,None,None,None,None,None
1,68355089,None,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,None,None,Cash,N,None,None,None,None,None,None
2,68341763,None,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,None,None,Cash,N,None,None,None,None,None,None
3,66310712,None,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,None,None,Cash,N,None,None,None,None,None,None
4,68476807,None,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,None,None,Cash,N,None,None,None,None,None,None


In [7]:
df.groupBy("loan_status").count().toPandas()

,loan_status,count
0,Fully Paid,1076751
1,Default,40
2,None,33
3,In Grace Period,8436
4,Does not meet the credit policy. Status:Fully ...,1988
5,Charged Off,268558
6,Oct-2015,1
7,Late (31-120 days),21467
8,Current,878317
9,Does not meet the credit policy. Status:Charge...,761


In [8]:
df = df.filter('loan_status in ("Fully Paid", "Charged Off")')

In [9]:
df.groupBy("loan_status").count().toPandas()

,loan_status,count
0,Fully Paid,1076751
1,Charged Off,268558


In [10]:
df = df.drop('id', 'pymnt_plan', 'hardship_flag', 'out_prncp', 'out_prncp_inv', 'policy_code')

In [11]:
def spark_df_shape(df):
    return df.count(), len(df.columns)

In [12]:
print(spark_df_shape(df))

(1345309, 145)


let's drop these columns, as it will not contribute to our goal.

In [13]:
to_drop = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()

In [14]:
to_drop

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1345309,0,0,0,0,0,0,0,0,85785,...,1339554,1339554,226,171,1311942,1311957,1311977,1311998,1312012,1312016


In [15]:
to_drop.sum().sort_values(ascending=False).head(50)

member_id                                     1345309
next_pymnt_d                                  1345086
orig_projected_additional_accrued_interest    1341548
hardship_last_payment_amount                  1339554
hardship_payoff_balance_amount                1339554
hardship_dpd                                  1339550
hardship_loan_status                          1339548
hardship_start_date                           1339547
hardship_end_date                             1339546
hardship_length                               1339545
payment_plan_start_date                       1339543
hardship_amount                               1339540
deferral_term                                 1339532
hardship_status                               1339521
hardship_reason                               1339513
hardship_type                                 1339504
sec_app_mths_since_last_major_derog           1338662
sec_app_revol_util                            1327004
sec_app_num_rev_accts       

In [16]:
to_drop.sum().sort_values(ascending=False)[50:].head(50)

open_rv_12m                   807699
open_rv_24m                   807693
total_bal_il                  807685
open_il_24m                   807678
open_il_12m                   807673
open_act_il                   807645
open_acc_6m                   807631
mths_since_last_delinq        678599
mths_since_recent_inq         174049
num_tl_120dpd_2m              117400
mo_sin_old_il_acct            105485
emp_title                      85785
emp_length                     78511
pct_tl_nvr_dlq                 67681
avg_cur_bal                    67546
num_tl_30dpd                   67527
num_bc_tl                      67526
num_rev_accts                  67526
num_tl_op_past_12m             67525
num_actv_rev_tl                67525
num_tl_90g_dpd_24m             67525
num_rev_tl_bal_gt_0            67525
total_rev_hi_lim               67522
num_il_tl                      67521
num_op_rev_tl                  67520
num_actv_bc_tl                 67518
num_accts_ever_120_pd          67516
t

The first 95 columns has lots of nulls so I will drop them

In [17]:
to_drop = list(to_drop.sum().sort_values(ascending=False).head(95).index)

In [18]:
print(to_drop)

['member_id', 'next_pymnt_d', 'orig_projected_additional_accrued_interest', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount', 'hardship_dpd', 'hardship_loan_status', 'hardship_start_date', 'hardship_end_date', 'hardship_length', 'payment_plan_start_date', 'hardship_amount', 'deferral_term', 'hardship_status', 'hardship_reason', 'hardship_type', 'sec_app_mths_since_last_major_derog', 'sec_app_revol_util', 'sec_app_num_rev_accts', 'sec_app_open_act_il', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_open_acc', 'sec_app_mort_acc', 'sec_app_inq_last_6mths', 'sec_app_fico_range_low', 'revol_bal_joint', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'verification_status_joint', 'dti_joint', 'annual_inc_joint', 'settlement_term', 'settlement_percentage', 'settlement_amount', 'settlement_date', 'settlement_status', 'debt_settlement_flag_date', 'desc', 'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_last_major_derog',

In [19]:
df = df.drop(*to_drop)

In [20]:
print((df.count(), len(df.columns)))

(1345309, 50)


In [21]:
df = df.na.drop()

In [22]:
print(spark_df_shape(df))

(1340812, 50)


In [23]:
to_drop = ['fico_range_low', 'funded_amnt_inv', 'funded_amnt', 'total_pymnt_inv', 'total_pymnt', 'installment', 'collection_recovery_fee', 'total_rec_prncp', 'last_fico_range_low']
df = df.drop(*to_drop)

In [24]:
print(spark_df_shape(df))

(1340812, 41)


In [25]:
mapping = {'Fully Paid': 1, 'Charged Off': 0}

mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])

df = df.withColumn("loan_is_paid", mapping_expr[col("loan_status")])

In [26]:
df.groupBy("loan_is_paid").count().toPandas()

,loan_is_paid,count
0,1,1074961
1,0,265851


In [27]:
df = df.drop('loan_status')

In [28]:
distinct_terms = [x.term for x in df.select('term').distinct().collect()]

In [29]:
distinct_terms

[' 36 months', ' 60 months']

In [30]:
mapping = dict(zip(distinct_terms, [36, 60]))
mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])
df = df.withColumn("term_months", mapping_expr[col("term")])

In [31]:
df.select("term_months").distinct().show()

+-----------+
|term_months|
+-----------+
|         60|
|         36|
+-----------+



In [32]:
df = df.drop('term')

In [33]:
df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- url: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- fico_range_high: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- initial_list_status: string (nullable = true)
 |-- total_rec_int: string (n

In [34]:
df.groupBy('home_ownership').count().toPandas()

,home_ownership,count
0,OWN,144179
1,RENT,532381
2,MORTGAGE,663782
3,ANY,283
4,OTHER,142
5,NONE,45


In [35]:
df = df.replace('NONE', 'ANY', 'home_ownership')

In [36]:
df.groupBy('home_ownership').count().toPandas()

,home_ownership,count
0,OWN,144179
1,RENT,532381
2,MORTGAGE,663782
3,ANY,328
4,OTHER,142


In [37]:
to_drop=['grade', 'issue_d', 'url', 'last_pymnt_d', 'last_credit_pull_d', 'zip_code', 'addr_state', 'earliest_cr_line']
df = df.drop(*to_drop)

In [38]:
df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- fico_range_high: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- initial_list_status: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- recoveries: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_fico_range_high: string (nullable = true)
 |-- collections_12_mths_ex_med: s

In [39]:
cols = ['annual_inc', 'dti', 'delinq_2yrs', 'fico_range_high', 'inq_last_6mths', 
        'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 
        'total_rec_late_fee', 'recoveries', 'last_pymnt_amnt', 'last_fico_range_high', 
        'collections_12_mths_ex_med', 'acc_now_delinq']
for col_name in cols:
    df = df.withColumn(col_name, col(col_name).cast('float'))

In [40]:
df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- fico_range_high: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- open_acc: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: float (nullable = true)
 |-- initial_list_status: string (nullable = true)
 |-- total_rec_int: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- recoveries: float (nullable = true)
 |-- last_pymnt_amnt: float (nullable = true)
 |-- last_fico_range_high: float (nullable = true)
 |-- collections_12_mths_ex_med: float (nullable 

In [41]:
df.limit(5).toPandas()

,loan_amnt,int_rate,sub_grade,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,fico_range_high,...,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,disbursement_method,debt_settlement_flag,loan_is_paid,term_months
0,3600.0,13.99,C4,MORTGAGE,55000.0,Not Verified,debt_consolidation,5.910000,0.0,679.0,...,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N,1,36
1,24700.0,11.99,C1,MORTGAGE,65000.0,Not Verified,small_business,16.059999,1.0,719.0,...,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N,1,36
2,20000.0,10.78,B4,MORTGAGE,63000.0,Not Verified,home_improvement,10.780000,0.0,699.0,...,Joint App,0.0,0.0,0.0,0.0,0.0,Cash,N,1,60
3,10400.0,22.45,F1,MORTGAGE,104433.0,Source Verified,major_purchase,25.370001,1.0,699.0,...,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N,1,60
4,11950.0,13.44,C3,RENT,34000.0,Source Verified,debt_consolidation,10.200000,0.0,694.0,...,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N,1,36


In [42]:
# Spark Pipeline
cat_features = ['sub_grade', 'verification_status', 'application_type', 'initial_list_status', 
                'purpose', 'home_ownership', 'disbursement_method', 'debt_settlement_flag']
assembler_inputs = list(set(df.columns) - set(cat_features)) + [feature + "_class_vec" for feature in cat_features]
assembler_inputs.remove('loan_is_paid')
assembler_inputs

['total_acc',
 'loan_amnt',
 'dti',
 'revol_util',
 'tax_liens',
 'pub_rec',
 'chargeoff_within_12_mths',
 'total_rec_int',
 'fico_range_high',
 'annual_inc',
 'open_acc',
 'last_pymnt_amnt',
 'delinq_2yrs',
 'delinq_amnt',
 'last_fico_range_high',
 'recoveries',
 'revol_bal',
 'acc_now_delinq',
 'term_months',
 'int_rate',
 'pub_rec_bankruptcies',
 'total_rec_late_fee',
 'inq_last_6mths',
 'collections_12_mths_ex_med',
 'sub_grade_class_vec',
 'verification_status_class_vec',
 'application_type_class_vec',
 'initial_list_status_class_vec',
 'purpose_class_vec',
 'home_ownership_class_vec',
 'disbursement_method_class_vec',
 'debt_settlement_flag_class_vec']

In [43]:
# Pipeline Stages List
stages = []

# Loop for StringIndexer and OHE for Categorical Variables
for features in cat_features:
    # Index Categorical Features
    string_indexer = StringIndexer(inputCol=features, outputCol=features + "_index")
    # One Hot Encode Categorical Features
    encoder = OneHotEncoder(inputCols=[string_indexer.getOutputCol()], outputCols=[features + "_class_vec"])
    # Append Pipeline Stages
    stages += [string_indexer, encoder]
    
assembler_final = VectorAssembler(inputCols=assembler_inputs, outputCol="features") 

stages += [assembler_final]

In [44]:
stages

[StringIndexer_746ac282e186,
 OneHotEncoder_fb5fe76b2cb1,
 StringIndexer_6ad93e3b29b0,
 OneHotEncoder_cc014fe82f92,
 StringIndexer_d919f21325a5,
 OneHotEncoder_b175e7e33995,
 StringIndexer_7ce1f840c1a8,
 OneHotEncoder_275334ab715b,
 StringIndexer_e701502d57df,
 OneHotEncoder_ef66d90c9edb,
 StringIndexer_e2e8ab6c3b3e,
 OneHotEncoder_fa56aa2e4a8f,
 StringIndexer_c243c9ddb932,
 OneHotEncoder_a2d7334a2a4a,
 StringIndexer_a3d96c225256,
 OneHotEncoder_37214ac0df2c,
 VectorAssembler_ee98ad8037d9]

In [45]:
# Set Pipeline
pipeline = Pipeline(stages=stages)

In [46]:
# Fit Pipeline to Data
pipeline_model = pipeline.fit(df)

In [47]:
# Transform Data using Fitted Pipeline
df = pipeline_model.transform(df)

In [48]:
# Preview Newly Transformed Data
df.limit(5).toPandas()

,loan_amnt,int_rate,sub_grade,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,fico_range_high,...,initial_list_status_class_vec,purpose_index,purpose_class_vec,home_ownership_index,home_ownership_class_vec,disbursement_method_index,disbursement_method_class_vec,debt_settlement_flag_index,debt_settlement_flag_class_vec,features
0,3600.0,13.99,C4,MORTGAGE,55000.0,Not Verified,debt_consolidation,5.910000,0.0,679.0,...,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,(1.0),"(13.0, 3600.0, 5.909999847412109, 29.700000762..."
1,24700.0,11.99,C1,MORTGAGE,65000.0,Not Verified,small_business,16.059999,1.0,719.0,...,(1.0),6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,(1.0),"(38.0, 24700.0, 16.059999465942383, 19.2000007..."
2,20000.0,10.78,B4,MORTGAGE,63000.0,Not Verified,home_improvement,10.780000,0.0,699.0,...,(1.0),2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,(1.0),"(18.0, 20000.0, 10.779999732971191, 56.2000007..."
3,10400.0,22.45,F1,MORTGAGE,104433.0,Source Verified,major_purchase,25.370001,1.0,699.0,...,(1.0),4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,(1.0),"(35.0, 10400.0, 25.3700008392334, 64.5, 0.0, 0..."
4,11950.0,13.44,C3,RENT,34000.0,Source Verified,debt_consolidation,10.200000,0.0,694.0,...,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.0, 1.0, 0.0, 0.0)",0.0,(1.0),0.0,(1.0),"(6.0, 11950.0, 10.199999809265137, 68.40000152..."


In [49]:
df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- fico_range_high: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- open_acc: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: float (nullable = true)
 |-- initial_list_status: string (nullable = true)
 |-- total_rec_int: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- recoveries: float (nullable = true)
 |-- last_pymnt_amnt: float (nullable = true)
 |-- last_fico_range_high: float (nullable = true)
 |-- collections_12_mths_ex_med: float (nullable 

In [50]:
df = df.select('features','loan_is_paid')
df.limit(20).toPandas()

,features,loan_is_paid
0,"(13.0, 3600.0, 5.909999847412109, 29.700000762...",1
1,"(38.0, 24700.0, 16.059999465942383, 19.2000007...",1
2,"(18.0, 20000.0, 10.779999732971191, 56.2000007...",1
3,"(35.0, 10400.0, 25.3700008392334, 64.5, 0.0, 0...",1
4,"(6.0, 11950.0, 10.199999809265137, 68.40000152...",1
5,"(27.0, 20000.0, 14.670000076293945, 84.5, 0.0,...",1
6,"(15.0, 20000.0, 17.610000610351562, 5.69999980...",1
7,"(23.0, 10000.0, 13.069999694824219, 34.5, 0.0,...",1
8,"(18.0, 8000.0, 34.79999923706055, 39.099998474...",1
9,"(24.0, 1400.0, 34.95000076293945, 67.199996948...",1


In [51]:
# Shuffle Data
df = df.orderBy(rand())

In [52]:
# Split Data into Train / Test Sets
train_data, test_data = df.randomSplit([.8, .2], seed=42)

In [53]:
print(spark_df_shape(df))

(1340812, 2)


In [54]:
# Number of Inputs or Input Dimensions
input_dim = len(train_data.select("features").first()[0])
input_dim

81

In [55]:
print(spark_df_shape(train_data))

(1073352, 2)


In [56]:
print(spark_df_shape(test_data))

(267460, 2)


In [92]:
# def create_model():
#     # Set up Deep Learning Model / Architecture
#     model = Sequential()
#     model.add(Dense(units=78, input_shape=(input_dim,), activation='relu'))
#     model.add(Dense(units=39,activation='relu'))
#     model.add(Dense(units=19,activation='relu'))
#     model.add(Dense(units=8,activation='relu'))
#     model.add(Dense(units=4,activation='relu'))
#     model.add(Dense(units=2,activation='softmax'))
#     return model

# def compile_model(model):
#     model.compile(loss='categorical_crossentrop', optimizer='adam', metrics=['accuracy'])
#     return model

def create_model():
    # Set up Deep Learning Model / Architecture
    model = Sequential()
    model.add(Dense(units=78, input_shape=(input_dim,), activation='relu'))
    model.add(Dense(units=39,activation='relu'))
    model.add(Dense(units=19,activation='relu'))
    model.add(Dense(units=8,activation='relu'))
    model.add(Dense(units=4,activation='relu'))
    model.add(Dense(units=1,activation='sigmoid'))
    return model

def compile_model(model):
    model.compile(loss='binary_crossentrop', optimizer='adam', metrics=['accuracy'])
    return model

In [93]:
model = create_model()
model = compile_model(model)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 78)                6396      
_________________________________________________________________
dense_25 (Dense)             (None, 39)                3081      
_________________________________________________________________
dense_26 (Dense)             (None, 19)                760       
_________________________________________________________________
dense_27 (Dense)             (None, 8)                 160       
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 5         
Total params: 10,438
Trainable params: 10,438
Non-trainable params: 0
__________________________________________________

In [73]:
train_data.groupBy("loan_is_paid").count().toPandas()

,loan_is_paid,count
0,1,860598
1,0,212754


In [74]:
test_data.groupBy("loan_is_paid").count().toPandas()

,loan_is_paid,count
0,1,214363
1,0,53097


In [99]:
adam = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(adam)

# Initialize SparkML Estimator and set all relevant properties
estimator = ElephasEstimator()
estimator.setFeaturesCol("features")
estimator.setLabelCol("loan_is_paid")
estimator.set_keras_model_config(model.to_yaml()) # Provide serialized Keras model
estimator.set_num_workers(1)                      # We just use one worker here. Feel free to adapt it.
estimator.set_epochs(10)
estimator.set_batch_size(512)
estimator.set_verbosity(1)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(True)
estimator.set_nb_classes(2)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous")
# estimator.set_loss("categorical_crossentrop")
estimator.set_loss("binary_crossentrop")
estimator.set_metrics(['accuracy'])

############################################################################################


# from elephas.spark_model import SparkModel

# spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
# spark_model.fit(rdd, epochs=5, batch_size=512, verbose=1, validation_split=0.1)

############################################################################################

# epochs = 40
# batch_size = 512
# nb_classes = 2
# estimator = ElephasEstimator(epochs=epochs, batch_size=batch_size, frequency='batch', mode='asynchronous', categorical=True, nb_classes=nb_classes)
# # estimator = ElephasEstimator(epochs=epochs, batch_size=batch_size, frequency='batch', mode='asynchronous', categorical=True)

# estimator.setFeaturesCol("features")
# estimator.setLabelCol("loan_is_paid")
# estimator.set_keras_model_config(model.to_yaml())
# estimator.set_verbosity(1)
# estimator.set_validation_split(0.1)
# adam = optimizers.Adam(lr=0.01)
# opt_conf = optimizers.serialize(adam)

# estimator.set_loss("categorical_crossentrop")
# estimator.set_metrics(['acc'])

ElephasEstimator_6638f9879e70

In [60]:
train_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- loan_is_paid: integer (nullable = true)



In [61]:
test_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- loan_is_paid: integer (nullable = true)



In [100]:
dl_pipeline = Pipeline(stages=[estimator])

In [101]:
fit_dl_pipeline = dl_pipeline.fit(train_data)

>>> Fit model
>>> Initialize workers
>>> Distribute load
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Process Process-8:
Traceback (most recent call last):
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/elephas/parameter/server.py", line 136, in start_flask_service
    threaded=self.threaded, use_reloader=self.use_reloader)
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/werkzeug/serving.py", line 1005, in inner
    fd=fd,
  File "/home/abdulrahman/anaconda3/envs/b

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 98.0 failed 1 times, most recent failure: Lost task 0.0 in stage 98.0 (TID 2289) (192.168.1.5 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/abdulrahman/anaconda3/envs/bigdata/lib/pyth

In [ ]:
pred_test = fit_dl_pipeline.transform(test_data)

In [ ]:
pred_test.limit(10).toPandas()

In [ ]:
pnl_test = pred_test.select("loan_is_paid", "prediction")

In [ ]:
pred_and_label_test = pnl_test.rdd.map(lambda row: (row["loan_is_paid"], row['prediction']))

In [ ]:
metrics_test = MulticlassMetrics(pred_and_label_test)

In [ ]:
print("\nTest Data Accuracy: {}".format(metrics_test.weightedPrecision))

In [ ]:
print("Test Data Confusion Matrix")
display(pnl_test.crosstab('loan_is_paid', 'prediction').toPandas())